In [1]:
import datetime
import numpy as np 
import time
import re
from datetime import datetime
import pickle
import string
import pandas as pd
from copy import deepcopy
import operator
import matplotlib.pyplot as plt
import os

import ast
from bs4 import BeautifulSoup
import urllib.request
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

import smtplib
from email.mime.text import MIMEText

In [2]:
def scrap_coin_gecko(symbol_to_match):
    
    url = f"https://www.coingecko.com/en/coins/recently_added"
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}

    r = requests.get(url, headers)
    html_doc = r.text
    soup = BeautifulSoup(html_doc, 'html.parser')

    soup_tag = soup.find_all("a", attrs={"class": "d-lg-none font-bold"})
    
    coin_symbol_list = []
    for element in soup_tag:
        
        list_letters = re.findall('([A-Z])', str(element))
        coin_symbol = "".join(list_letters)
        
        coin_symbol_list.append(coin_symbol)
    
    is_symbol_same = coin_symbol_list[0] == symbol_to_match
    
    if not is_symbol_same:
        print("coin gecko", coin_symbol_list[0])

    return is_symbol_same, coin_symbol_list[0], coin_symbol_list

In [3]:
def scrap_coinmartket(symbol_to_match):
    
    url = f"https://coinmarketcap.com/new/"
    
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}

    r = requests.get(url, headers)
    html_doc = r.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    
    soup_tag = soup.find_all("img", attrs={"class": "coin-logo"})

    coin_symbol_list = []
    for element in soup_tag:
        
        list_letters = re.findall('([A-Z])', str(element))
        coin_symbol = "".join(list_letters)
        
        coin_symbol_list.append(coin_symbol)

    is_symbol_same = coin_symbol_list[0] == symbol_to_match
    
    if not is_symbol_same:
        print("coin market", coin_symbol_list[0])
    
    return is_symbol_same, coin_symbol_list[0], coin_symbol_list

In [16]:
bool_coin_market = True
bool_coin_gecko = True

In [17]:
symbol_coin_market = "DRAC"
symbol_coin_gecko = "GUCCINU"

In [ ]:
while bool_coin_gecko and bool_coin_market:
    
    time.sleep(1)
    
    bool_coin_market, symbol_marketcap, symbol_marketcap_list = scrap_coinmartket(symbol_coin_market)
    bool_coin_gecko, symbol_gecko, symbol_gecko_list = scrap_coin_gecko(symbol_coin_gecko)

    if not bool_coin_market:
        is_already_in = symbol_marketcap in symbol_gecko_list
    
        if is_already_in:
            bool_coin_market = True
            symbol_coin_market = symbol_marketcap
            
    if not bool_coin_gecko:
        is_already_in = symbol_gecko in symbol_marketcap_list
    
        if is_already_in:
            bool_coin_gecko = True
            symbol_coin_gecko = symbol_gecko
    
for _ in range(50):
    os.system("beep -f 555 -l 460")
    time.sleep(0.1)
